Importing libries

In [8]:
import pandas as pd
import numpy as np

Uploading the FULL dataframe

In [9]:
dfFull = pd.read_excel("Faturamento_Truck.xlsx", sheet_name="FULL", header=3)

In [10]:
dfFull

,Unnamed: 0,NS,EMPRESA,ENDEREÇO,SITE,UF,PAIS,CEP,FABRICA,CC,...,MODELO,NS.1,IP,SSB - FILA,TP IMPR,CONT INIC,CONT FINAL,PAG IMPR,VALOR PAG,VALOR TOTAL
0,NaN,5024865648855,Truck,Avenida Truck Senna,SP,SAO PAULO,Brazil,0000-000,TRUCK,98756.0,...,PRINTERME,5.024866e+12,0.0.0.0,IMPRESSORATRUCK,Color,22788.0,22995.0,207,0.19,39.3300
1,NaN,5027469457XP2,Truck,Avenida Truck Senna,SP,SAO PAULO,Brazil,0000-001,TRUCK,98756.0,...,PRINTERME,5.024866e+12,0.0.0.1,IMPRESSORATRUCK,Color,19188.0,19362.0,174,0.19,33.0600
2,NaN,5027469457XRP,Truck,Avenida Truck Senna,SP,SAO PAULO,Brazil,0000-002,TRUCK,98756.0,...,PRINTERME,5.024866e+12,0.0.0.2,IMPRESSORATRUCK,Color,25992.0,26331.0,339,0.19,64.4100
3,NaN,502714945KHGB,Truck,Avenida Truck Senna,SP,SAO PAULO,Brazil,0000-003,TRUCK,98756.0,...,PRINTERME,5.024866e+12,0.0.0.3,IMPRESSORATRUCK,Color,14386.0,14386.0,0,0.19,0.0000
4,NaN,502714945KHM3,Truck,Avenida Truck Senna,SP,SAO PAULO,Brazil,0000-004,TRUCK,98756.0,...,PRINTERME,5.024866e+12,0.0.0.4,IMPRESSORATRUCK,Color,6443.0,6925.0,482,0.19,91.5800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,NaN,NaN,TOTAL EM BILHETAGEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,735651,NaN,24758.8904
220,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CUSTO FIXO,NaN,10000.0000
221,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ORÇAMENTOS,NaN,0.0000
222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TONER REC,NaN,7000.0000


Uploading the TONER REC dataframe

In [11]:
dfTonerRec = pd.read_excel("Faturamento_Truck.xlsx", sheet_name="TONER REC", header=6)

Uploading the Valor Fixo

In [12]:
dfValorFixo = pd.read_excel("Faturamento_Truck.xlsx", sheet_name="RESUMO - FATURAMENTO")

Delete columns with NaN columns in all cells

In [13]:
colunDel = dfFull.dropna(axis=1, how="all")

Delete lines with more than 5 NaN values

In [14]:
lineDel = colunDel.dropna(axis=0, thresh=5)

Replace NaN values with the last filled value in the line before

In [15]:
rplNullValue = lineDel.replace(np.nan, method="ffill")

Transforming column CC as int type

In [16]:
rplNullValue["CC"] = rplNullValue["CC"].astype("int")

Selecting columns with the valuable information

In [17]:
selectColumns = rplNullValue[["CC", "VALOR TOTAL"]]

Removing 0.0 values using a filter

In [18]:
filter = selectColumns["VALOR TOTAL"] != 0
selectColumns = selectColumns[filter]

Getting the fixed value to divide to all CC's

In [19]:
valorFixo = dfValorFixo.iat[6,5]

Getting the Full dataframe lenth

In [20]:
sizeFullDf = len(selectColumns)

Dividing the fixed value to all CC equally

In [21]:
rateioValorFixo = valorFixo / sizeFullDf

Adding fixed value into the dataframe full

In [22]:
selectColumns["RateioFixo"] = rateioValorFixo

Calculating the sum of total value with fixed value

In [23]:
selectColumns["TotalRateio"] = selectColumns["VALOR TOTAL"] + selectColumns["RateioFixo"]

Grouping dataframe full by CC

In [24]:
rateioAgrupado = selectColumns.groupby("CC").sum().reset_index()

Separating dataframe full by CC & TotalRateio

In [25]:
ccTotalRateio = rateioAgrupado[["CC", "TotalRateio"]]

Rounding column TotalRateio to 2 decimals

In [26]:
rateioFinal = ccTotalRateio.round({'TotalRateio': 2})

In [27]:
rateioFinal

,CC,TotalRateio
0,11111,9621.86
1,48899,820.93
2,55555,2236.68
3,56489,3984.68
4,66666,1100.11
5,88888,2164.82
6,97788,4267.24
7,98756,10232.38
8,99999,330.19


In [28]:
rateioFinal.to_csv("RateioFinal08.csv", header=False, index=False, sep=" ", float_format="%.2f")

In [29]:
np.sum(rateioFinal)

CC             624151.00
TotalRateio     34758.89
dtype: float64

------------------------------------------------------------------------------------------------------

Initializing Toner Rec analysis

In [30]:
dfTonerRec

,Supplier ID,Lexmark Customer Number,Lexmark Product Code,Product Part Desc,Quantity Shipped,Ship Date,Printer Serial Number,Order Number,Order Create Date,FABRICA,SITE,C/C,Valor
0,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN
1,LA Lexmark Direct,All,70CBXK0,Lxk CS/CX510 Blk Rtn 8K Crtg,1.0,2021-04-05 19:15:21,50272394661BV,1-375006741969,2021-03-25 18:57:18,TRUCK,SP,0.0,49.52625
2,LA Lexmark Direct,All,70CBXK0,Lxk CS/CX510 Blk Rtn 8K Crtg,1.0,2021-04-05 19:15:44,50274994582VH,1-375314039060,2021-03-29 20:03:22,TRUCK,SP,98756.0,49.52625
3,LA Lexmark Direct,All,70C8XC0,Lxk CS/CX510 Cyn Rtn 4K Crtg,1.0,2021-04-07 15:15:22,502714945KHD9,1-375528534576,2021-03-31 18:33:21,TRUCK,SP,88888.0,49.52625
4,LA Lexmark Direct,All,52DBX0E,52DBX0E Black CRTG Extra CORP,1.0,2021-04-06 22:30:51,40634C6600F37,1-375851892131,2021-04-05 12:39:21,TRUCK,SP,11111.0,49.52625
5,LA Lexmark Direct,All,52DBX0E,52DBX0E Black CRTG Extra CORP,1.0,2021-04-06 22:15:17,40634C6600DK3,1-375878558570,2021-04-05 20:33:23,TRUCK,SP,97788.0,49.52625
6,LA Lexmark Direct,All,70CBXY0,Lxk CS/CX510 Yel Rtn 4K Crtg,1.0,2021-04-08 19:30:34,50272394661BV,1-375952763855,2021-04-06 14:09:26,TRUCK,SP,0.0,49.52625
7,LA Lexmark Direct,All,52D0Z0E,MS71/81x Black IU CORP,1.0,2021-04-06 22:31:50,40634C6600DKG,1-375955092830,2021-04-06 14:27:19,TRUCK,SP,97788.0,49.52625
8,LA Lexmark Direct,All,52DBX0E,52DBX0E Black CRTG Extra CORP,1.0,2021-04-13 14:15:26,40634C6600F6K,1-375968581297,2021-04-06 17:21:18,TRUCK,SP,11111.0,49.52625
9,LA Lexmark Direct,All,52DBX0E,52DBX0E Black CRTG Extra CORP,1.0,2021-04-13 14:01:17,40634C6600FLB,1-375971730415,2021-04-06 17:57:22,TRUCK,SP,11111.0,49.52625


Deleting lines with more than 5 NaN values

In [31]:
lineDelRec = dfTonerRec.dropna(axis=0, thresh=5)

In [32]:
lineDelRec

,Supplier ID,Lexmark Customer Number,Lexmark Product Code,Product Part Desc,Quantity Shipped,Ship Date,Printer Serial Number,Order Number,Order Create Date,FABRICA,SITE,C/C,Valor
1,LA Lexmark Direct,All,70CBXK0,Lxk CS/CX510 Blk Rtn 8K Crtg,1.0,2021-04-05 19:15:21,50272394661BV,1-375006741969,2021-03-25 18:57:18,TRUCK,SP,0.0,49.52625
2,LA Lexmark Direct,All,70CBXK0,Lxk CS/CX510 Blk Rtn 8K Crtg,1.0,2021-04-05 19:15:44,50274994582VH,1-375314039060,2021-03-29 20:03:22,TRUCK,SP,98756.0,49.52625
3,LA Lexmark Direct,All,70C8XC0,Lxk CS/CX510 Cyn Rtn 4K Crtg,1.0,2021-04-07 15:15:22,502714945KHD9,1-375528534576,2021-03-31 18:33:21,TRUCK,SP,88888.0,49.52625
4,LA Lexmark Direct,All,52DBX0E,52DBX0E Black CRTG Extra CORP,1.0,2021-04-06 22:30:51,40634C6600F37,1-375851892131,2021-04-05 12:39:21,TRUCK,SP,11111.0,49.52625
5,LA Lexmark Direct,All,52DBX0E,52DBX0E Black CRTG Extra CORP,1.0,2021-04-06 22:15:17,40634C6600DK3,1-375878558570,2021-04-05 20:33:23,TRUCK,SP,97788.0,49.52625
6,LA Lexmark Direct,All,70CBXY0,Lxk CS/CX510 Yel Rtn 4K Crtg,1.0,2021-04-08 19:30:34,50272394661BV,1-375952763855,2021-04-06 14:09:26,TRUCK,SP,0.0,49.52625
7,LA Lexmark Direct,All,52D0Z0E,MS71/81x Black IU CORP,1.0,2021-04-06 22:31:50,40634C6600DKG,1-375955092830,2021-04-06 14:27:19,TRUCK,SP,97788.0,49.52625
8,LA Lexmark Direct,All,52DBX0E,52DBX0E Black CRTG Extra CORP,1.0,2021-04-13 14:15:26,40634C6600F6K,1-375968581297,2021-04-06 17:21:18,TRUCK,SP,11111.0,49.52625
9,LA Lexmark Direct,All,52DBX0E,52DBX0E Black CRTG Extra CORP,1.0,2021-04-13 14:01:17,40634C6600FLB,1-375971730415,2021-04-06 17:57:22,TRUCK,SP,11111.0,49.52625
10,LA Lexmark Direct,All,52DBX0E,52DBX0E Black CRTG Extra CORP,1.0,2021-04-13 14:01:17,40634C6600F1L,1-375976609740,2021-04-06 18:39:16,TRUCK,SP,11111.0,49.52625


Changind column C/C type to int

In [33]:
lineDelRec['C/C']  = lineDelRec['C/C'].astype('int')

<ipython-input-33-7fab81dcf94c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lineDelRec['C/C']  = lineDelRec['C/C'].astype('int')


Here I am verifying all CC with 0 values, getting the SITE column value and replacing the 0 by a not null SITE CC.

In [34]:
for linha in range(len(lineDelRec)):
    if lineDelRec['C/C'].iloc[linha] in [0]:
        x = lineDelRec['SITE'].iloc[linha]
        for linha2 in range(len(lineDelRec)):
            if lineDelRec['SITE'].iloc[linha2] in [x] and lineDelRec['C/C'].iloc[linha2] != 0 :
                y = lineDelRec['C/C'].iloc[linha2]
                lineDelRec['C/C'].iloc[linha] = y

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [35]:
lineDelRec

,Supplier ID,Lexmark Customer Number,Lexmark Product Code,Product Part Desc,Quantity Shipped,Ship Date,Printer Serial Number,Order Number,Order Create Date,FABRICA,SITE,C/C,Valor
1,LA Lexmark Direct,All,70CBXK0,Lxk CS/CX510 Blk Rtn 8K Crtg,1.0,2021-04-05 19:15:21,50272394661BV,1-375006741969,2021-03-25 18:57:18,TRUCK,SP,97788,49.52625
2,LA Lexmark Direct,All,70CBXK0,Lxk CS/CX510 Blk Rtn 8K Crtg,1.0,2021-04-05 19:15:44,50274994582VH,1-375314039060,2021-03-29 20:03:22,TRUCK,SP,98756,49.52625
3,LA Lexmark Direct,All,70C8XC0,Lxk CS/CX510 Cyn Rtn 4K Crtg,1.0,2021-04-07 15:15:22,502714945KHD9,1-375528534576,2021-03-31 18:33:21,TRUCK,SP,88888,49.52625
4,LA Lexmark Direct,All,52DBX0E,52DBX0E Black CRTG Extra CORP,1.0,2021-04-06 22:30:51,40634C6600F37,1-375851892131,2021-04-05 12:39:21,TRUCK,SP,11111,49.52625
5,LA Lexmark Direct,All,52DBX0E,52DBX0E Black CRTG Extra CORP,1.0,2021-04-06 22:15:17,40634C6600DK3,1-375878558570,2021-04-05 20:33:23,TRUCK,SP,97788,49.52625
6,LA Lexmark Direct,All,70CBXY0,Lxk CS/CX510 Yel Rtn 4K Crtg,1.0,2021-04-08 19:30:34,50272394661BV,1-375952763855,2021-04-06 14:09:26,TRUCK,SP,97788,49.52625
7,LA Lexmark Direct,All,52D0Z0E,MS71/81x Black IU CORP,1.0,2021-04-06 22:31:50,40634C6600DKG,1-375955092830,2021-04-06 14:27:19,TRUCK,SP,97788,49.52625
8,LA Lexmark Direct,All,52DBX0E,52DBX0E Black CRTG Extra CORP,1.0,2021-04-13 14:15:26,40634C6600F6K,1-375968581297,2021-04-06 17:21:18,TRUCK,SP,11111,49.52625
9,LA Lexmark Direct,All,52DBX0E,52DBX0E Black CRTG Extra CORP,1.0,2021-04-13 14:01:17,40634C6600FLB,1-375971730415,2021-04-06 17:57:22,TRUCK,SP,11111,49.52625
10,LA Lexmark Direct,All,52DBX0E,52DBX0E Black CRTG Extra CORP,1.0,2021-04-13 14:01:17,40634C6600F1L,1-375976609740,2021-04-06 18:39:16,TRUCK,SP,11111,49.52625


Selecting columns with the relevant information

In [36]:
lineDelRec = lineDelRec[['C/C', 'Valor']]

In [37]:
lineDelRec

,C/C,Valor
1,97788,49.52625
2,98756,49.52625
3,88888,49.52625
4,11111,49.52625
5,97788,49.52625
6,97788,49.52625
7,97788,49.52625
8,11111,49.52625
9,11111,49.52625
10,11111,49.52625


Getting Fixed value from Toner Rec

In [38]:
valorFixoRec = dfValorFixo.iat[7,5]

Getting the full dataframe lenth

In [39]:
lineDelRec = lineDelRec.groupby('C/C').sum().reset_index()

In [40]:
lineDelRec

,C/C,Valor
0,11111,841.94625
1,55555,297.15750
2,56489,297.15750
3,66666,49.52625
4,88888,247.63125
5,97788,594.31500
6,98756,297.15750
7,99999,148.57875


In [41]:
np.sum(lineDelRec)

C/C      575252.00
Valor      2773.47
dtype: float64

In [42]:
lineDelRec.to_csv("RateioFinalRec.csv", header=False, index=False, sep=" ", float_format="%.2f")